## Challenge Description

Company XYZ is a worldwide e-commerce site with localized versions of the site.
A data scientist at XYZ noticed that Spain-based users have a much higher conversion rate than
any other Spanish-speaking country. She therefore went and talked to the international team in
charge of Spain And LatAm to see if they had any ideas about why that was happening.

Spain and LatAm country manager suggested that one reason could be translation. All Spanish-
speaking countries had the same translation of the site which was written by a Spaniard. They agreed to try a test where each country would have its one translation written by a local. That is,
Argentinian users would see a translation written by an Argentinian, Mexican users by a Mexican
and so on. Obviously, nothing would change for users from Spain.
After they run the test however, they are really surprised cause the test is negative. I.e., it
appears that the non-localized translation was doing better!

##### You are asked to:
1. Confirm that the test is actually negative. That is, it appears that the old version of the site with just one translation across Spain and LatAm performs better
2. Explain why that might be happening. Are the localized translations really worse?
3. If you identified what was wrong, design an algorithm that would return FALSE if the same problem is happening in the future and TRUE if everything is good and the results can be trusted.

##### For A/B Testing, a crucial assumption is that the only difference between test and control should be the feature we are testing. 
##### Sample size should be large enough, test and control distribution should be the same

In [1]:
# Import pandas to read the data
import pandas as pd
pd.set_option('display.max_column', 20)
pd.set_option('display.width', 350)

In [5]:
# Read data
user = pd.read_csv('user_table.csv')
test = pd.read_csv('test_table.csv')

print('User Data', user.shape)
print('Test Data', test.shape)

User Data (452867, 4)
Test Data (453321, 9)


In [7]:
user.head()

,user_id,sex,age,country
0,765821,M,20,Mexico
1,343561,F,27,Nicaragua
2,118744,M,23,Colombia
3,987753,F,27,Venezuela
4,554597,F,20,Spain


In [8]:
test.head()

,user_id,date,source,device,browser_language,ads_channel,browser,conversion,test
0,315281,2015-12-03,Direct,Web,ES,NaN,IE,1,0
1,497851,2015-12-04,Ads,Web,ES,Google,IE,0,1
2,848402,2015-12-04,Ads,Web,ES,Facebook,Chrome,0,0
3,290051,2015-12-03,Ads,Mobile,Other,Facebook,Android_App,0,1
4,548435,2015-11-30,Ads,Web,ES,Google,FireFox,0,1


In [11]:
# Check if the user_ids are unique
print(test['user_id'].nunique() == len(test['user_id']))
print(len(test['user_id']))

True
453321


In [12]:
print(user['user_id'].nunique() == len(user['user_id']))
print(len(user['user_id']))

True
452867


##### From the results above we can find that the user's number is not consistant between two table. 
Just try to get the common data from both tables.

In [17]:
# Inner join two tables
merge = user.merge(test, on=['user_id'])
merge.head()

,user_id,sex,age,country,date,source,device,browser_language,ads_channel,browser,conversion,test
0,765821,M,20,Mexico,2015-12-02,Ads,Mobile,ES,Yahoo,Android_App,0,1
1,343561,F,27,Nicaragua,2015-12-04,Ads,Web,ES,Facebook,Safari,0,0
2,118744,M,23,Colombia,2015-11-30,Ads,Mobile,ES,Facebook,Android_App,0,1
3,987753,F,27,Venezuela,2015-12-04,SEO,Web,ES,NaN,IE,0,1
4,554597,F,20,Spain,2015-12-04,Direct,Web,ES,NaN,Chrome,0,0


In [19]:
# Check the datatype for merge dataset
merge.info()
merge.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 452867 entries, 0 to 452866
Data columns (total 12 columns):
user_id             452867 non-null int64
sex                 452867 non-null object
age                 452867 non-null int64
country             452867 non-null object
date                452867 non-null object
source              452867 non-null object
device              452867 non-null object
browser_language    452867 non-null object
ads_channel         181693 non-null object
browser             452867 non-null object
conversion          452867 non-null int64
test                452867 non-null int64
dtypes: int64(4), object(8)
memory usage: 44.9+ MB


,user_id,age,conversion,test
count,452867.000000,452867.000000,452867.000000,452867.000000
mean,499944.805166,27.130740,0.049560,0.476462
std,288676.264784,6.776678,0.217034,0.499446
min,1.000000,18.000000,0.000000,0.000000
25%,249819.000000,22.000000,0.000000,0.000000
50%,500019.000000,26.000000,0.000000,0.000000
75%,749543.000000,31.000000,0.000000,1.000000
max,1000000.000000,70.000000,1.000000,1.000000


In [20]:
# convert the date to datetime format
merge.date = pd.to_datetime(merge.date)

In [21]:
merge.info()
merge.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 452867 entries, 0 to 452866
Data columns (total 12 columns):
user_id             452867 non-null int64
sex                 452867 non-null object
age                 452867 non-null int64
country             452867 non-null object
date                452867 non-null datetime64[ns]
source              452867 non-null object
device              452867 non-null object
browser_language    452867 non-null object
ads_channel         181693 non-null object
browser             452867 non-null object
conversion          452867 non-null int64
test                452867 non-null int64
dtypes: datetime64[ns](1), int64(4), object(7)
memory usage: 44.9+ MB


,user_id,age,conversion,test
count,452867.000000,452867.000000,452867.000000,452867.000000
mean,499944.805166,27.130740,0.049560,0.476462
std,288676.264784,6.776678,0.217034,0.499446
min,1.000000,18.000000,0.000000,0.000000
25%,249819.000000,22.000000,0.000000,0.000000
50%,500019.000000,26.000000,0.000000,0.000000
75%,749543.000000,31.000000,0.000000,1.000000
max,1000000.000000,70.000000,1.000000,1.000000


#### Q1：Confirm that the test is actually negative. That is, it appears that the old version of the site with just one translation across Spain and LatAm performs better 